# Vector Indexes

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY}")
print(f"OpenAI URL: {OPENAI_URL}")



## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

## Collection with HNSW index (default)

[HNSW params](https://weaviate.io/developers/weaviate/config-refs/schema/vector-index#hnsw-index-parameters)

In [ ]:
from weaviate.classes.config import Configure, VectorDistances

client.collections.delete("IndexExample")

# Create a collection here - with Weaviate as a vectorizer
client.collections.create(
    name="IndexExample",

    vector_config=[
        Configure.Vectors.text2vec_ollama(
            name="main_vector",

            model="nomic-embed-text",
            source_properties=['title', 'text'],

            # HSNW example 
            vector_index_config=Configure.VectorIndex.hnsw()
        )
    ],
)

## Collection with Flat index

[Flat params](https://weaviate.io/developers/weaviate/config-refs/schema/vector-index#flat-indexes)

In [ ]:
from weaviate.classes.config import Configure, VectorDistances

client.collections.delete("IndexExample")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="IndexExample",

    vector_config=[
        Configure.Vectors.text2vec_ollama(
            name="main_vector",

            model="nomic-embed-text",
            source_properties=['title', 'text'],

            # Flat example
            vector_index_config=Configure.VectorIndex.flat(
                # distance_metric=VectorDistances.COSINE, # optional
                vector_cache_max_objects=100_000,
            ),
        ),
    ],
)

## Collection with Dynamic index

[Dynamic params](https://weaviate.io/developers/weaviate/config-refs/schema/vector-index#dynamic-index-parameters)

In [ ]:
from weaviate.classes.config import Configure

client.collections.delete("IndexExample")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="IndexExample",

    vector_config=[
        Configure.Vectors.text2vec_ollama(
            name="main_vector",

            model="nomic-embed-text",
            source_properties=['title', 'text'],

            # Dynamic example
            vector_index_config=Configure.VectorIndex.dynamic(
                threshold=10_000, # when to switch to HNSW

                flat=Configure.VectorIndex.flat(
                    vector_cache_max_objects=100_000,
                    # note: can also include a quantizer
                    quantizer=Configure.VectorIndex.Quantizer.bq()
                ),

                hnsw=Configure.VectorIndex.hnsw(
                    max_connections=32, # optional
                    # note: the quantizer can be different between flat and hnsw
                    quantizer=Configure.VectorIndex.Quantizer.pq()
                ),
            ),
        )
    ],
)

## The rest is the same

In [ ]:
# comment this out if you want to import 25k objects to your collection to test it
# from data_loader import import_wiki_data
# import_wiki_data(client, "IndexExample", 25_000)

In [ ]:
# index_example = client.collections.get("IndexExample")
# index_example.aggregate.over_all()

## Clean up

In [ ]:
client.collections.delete("IndexExample")

## Close the client

In [ ]:
client.close()